In [ ]:
####################################
#
#  ADD THIS TO EVERY COLAB FILE!
#
####################################

#!pip install -q import-ipynb

from google.colab import drive
drive.mount('/content/drive')

#import import_ipynb
import drive.Shareddrives.GPTJ.project.settings as settings

PATH_PROJECT = settings.PATH_PROJECT
PATH_DATA = settings.PATH_DATA

! cd $PATH_PROJECT && pip install -q -r requirements.txt

!nvidia-smi

Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143 kB 5.3 MB/s 
     |████████████████████████████████| 55.9 MB 288 kB/s 
     |████████████████████████████████| 717 kB 69.0 MB/s 
     |████████████████████████████████| 163 kB 66.4 MB/s 
     |████████████████████████████████| 7.6 MB 61.0 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 120 kB 62.4 MB/s 
     |████████████████████████████████| 99 kB 10.1 MB/s 
Sat Oct  8 20:19:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|   

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, GPT2TokenizerFast, TrainingArguments, Trainer, GPT2LMHeadModel, GPTNeoForCausalLM

In [ ]:
# MODEL

#model_name = 'EleutherAI/gpt-neo-1.3B' # CUDA out of memory.

model_name = 'gpt2' # GPU vRAM 3495MiB - 3.5G
#model_name = 'gpt2-medium' # GPU vRAM 7389MiB - 7.3G
#model_name = 'gpt2-large' # GPU vRAM 14797MiB - 14.8G

#model_name = 'gpt2-xl' # CUDA out of memory.


In [ ]:
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)

# download
# tokenizer = GPT2Tokenizer.from_pretrained(
tokenizer = GPT2TokenizerFast.from_pretrained(
    model_name, 
    bos_token='<|startoftext|>',
    eos_token='<|endoftext|>', 
    pad_token='<|pad|>'
)
model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
model.resize_token_embeddings(len(tokenizer))

# torch.save(tokenizer, os.path.join(PATH_DATA, model_name, 'tokenizer'))
# torch.save(model, os.path.join(PATH_DATA, model_name, 'model'))

# load
# model = torch.load(os.path.join(PATH_DATA, model_name, 'model'))
# tokenizer = torch.load(os.path.join(PATH_DATA, model_name, 'tokenizer'))

In [ ]:
descriptions = pd.read_csv(PATH_PROJECT + 'netflix_titles.csv')['description']
descriptions = descriptions[:100]

max_length = max([len(tokenizer.encode(description)) for description in descriptions])
max_length, descriptions

(56, 0                           As her father nears the end.
 1      As her father nears the end of his life, filmm...
 2      After crossing paths at a party, a Cape Town t...
 3      To protect his family from a powerful drug lor...
 4      Feuds, flirtations and toilet talk go down amo...
                              ...                        
 995    Abandoned by her family, young single mother A...
 996    In 1974, a rural town in Anatolia gets its fir...
 997    Truth and illusion blurs when a homeless amnes...
 998    Using innovative technology, this docuseries e...
 999    Journalists and fans await Ma Anand Sheela as ...
 Name: description, Length: 1000, dtype: object)

In [ ]:
class NetflixDataset(Dataset):

    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer(
                '<|startoftext|>' + txt + '<|endoftext|>',
                truncation=True,
                max_length=max_length, 
                padding='max_length'
            )
            
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)

            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)

            # label = [tokenizer.decode(x).strip() for x in encodings_dict['input_ids']]
            # self.labels.append(label)


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = NetflixDataset(descriptions, tokenizer, max_length)

print(len(dataset))

train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

print('train_dataset', len(train_dataset))
print('val_dataset', len(val_dataset))
train_dataset[0]

1000
train_dataset 800
val_dataset 200


(tensor([50257,  5886,  6551, 12622, 30768,   422,   262,   995,  1642, 38748,
           292,   287,   465,  2802,   338, 46400, 15647,  1566,   257,   649,
         44511, 20385,   287,   290, 10114,    82,   465, 15133,    13, 50256,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]))

In [ ]:
! pip install -q datasets evaluate

In [ ]:
# Here I will pass the output directory where 
# the model predictions and checkpoints will be stored, 
# batch sizes for the training and validation steps, 
# and warmup_steps to gradually increase the learning rate
training_args = TrainingArguments(
    output_dir=os.path.join(PATH_DATA, model_name, 'partial'),
    logging_dir=os.path.join(PATH_DATA, model_name, 'logs'),

    logging_steps=5000,
    save_steps=5000,  

    num_train_epochs=5,
                                     
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,

    warmup_steps=0,
    weight_decay=0.01,

    evaluation_strategy='epoch', # -> calls on_epoch_end()
)

from transformers import TrainerCallback
from copy import deepcopy

# https://huggingface.co/transformers/v4.0.1/_modules/transformers/trainer_callback.html

class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        print('\n---')
        print('on_epoch_end()')
        print('state', state)
        print('control', control)
        print('')
        current_epoch = state.epoch
        previous_epoch = current_epoch -1


        # print('control.should_evaluate', control.should_evaluate)
        # if control.should_evaluate:
        #     control_copy = deepcopy(control)
        #     train = self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix='train')
        #     print('  train', train)
        #     eval = self._trainer.evaluate() #eval_dataset=self._trainer.val_dataset, metric_key_prefix='eval')
        #     print('  eval', eval)
        #     return control_copy

import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    # This custom collate function is necessary 
    # to built batches of data
    data_collator=lambda data: {
        'input_ids': torch.stack([f[0] for f in data]),       
        'attention_mask': torch.stack([f[1] for f in data]),
        'labels': torch.stack([f[0] for f in data])
    },
    # compute_metrics=compute_metrics,
)
# Start training process!
# trainer.add_callback(CustomCallback(trainer)) 
train = trainer.train()
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300


Epoch,Training Loss,Validation Loss
1,No log,2.068487
2,No log,2.091509
3,No log,2.120424


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


{'eval_loss': 2.120424270629883,
 'eval_runtime': 1.0222,
 'eval_samples_per_second': 195.658,
 'eval_steps_per_second': 24.457,
 'epoch': 3.0}

In [ ]:
# train.metrics
# trainer.predict(test_dataset=val_dataset, metric_key_prefix="val")

In [ ]:
# trainer.save_model(os.path.join(PATH_DATA, model_name, 'model-trained'))

In [ ]:

# Start every description with a special BOS token
# generated = tokenizer('<|startoftext|>', return_tensors='pt').input_ids.cuda()

# Start every description with a special BOS token
# prompt = 'mov ebp var_A dx' # mov edx ebp var_E'
# prompt = 'The key benefits of TF-IDF are'
prompt = 'Four women'
generated = tokenizer(prompt, return_tensors='pt').input_ids.cuda()

# Generate 3 movie descriptions
sample_outputs = model.generate(
    generated, 
    # Use sampling instead of greedy decoding 
    do_sample=True, 
    # Keep only top 50 token with the highest probability
    top_k=50, 
    # Maximum sequence length
    max_length=50,
    #max_new_tokens=100,
    # Keep only the most probable tokens with cumulative probability of 95%
    top_p=0.95, 
    # Changes randomness of generated sequences to 1.9
    temperature=1,
    # Number of sequences to generate                 
    num_return_sequences=5
)

# Print generated descriptions
for i, sample_output in enumerate(sample_outputs): 
    print('{}: {}'.format(i, tokenizer.decode(sample_output, skip_special_tokens=True)).replace('\n', ' '))

# completion = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
# print(completion)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Four women and children live in the city of Istanbul, where their families face challenges facing unemployment, poverty and political oppression.
1: Four women who slept with their boyfriends over the last five years have accused their ex-lover of cheating on them.
2: Four women and their families are denied a fair shake during a visit by the founder and the director of a prestigious private school, who leads them to believe that their children's magical abilities might be a gift from God.
3: Four women go on the run from a murderous mob with illegal weapons in Nigeria after fleeing in the face of an Indian mob, only to find that there's an even bigger threat.
4: Four women on Tinder have been caught cheating on men and taking up too much space on the dating scene, according to a new survey.


In [ ]:
!nvidia-smi

Sat Oct  8 20:30:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    40W /  70W |  10040MiB / 15109MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------